### <b>Image augmentation for INPUT: <b style="color:red">128x128</b></b>

In [1]:
import os
import cv2
import numpy as np
import shutil

In [2]:
class Augmentation:
    def __init__(self, random_bound=0.5):
        self.random_bound = random_bound
    
    def augment_image(self, image, rotate=True):
        if np.random.rand() > self.random_bound:
            image = cv2.flip(image, 1)
        if rotate:
            angle = np.random.choice([90, 180, 270])
            height, width = image.shape[:2]
            center = (width // 2, height // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
            image = cv2.warpAffine(image, rotation_matrix, (width, height))
        factor = np.random.uniform(1.0, 1.05)
        image = cv2.convertScaleAbs(image, alpha=factor, beta=0)
        return image
    
    def augment_image_file(self, path_to_input_image, path_to_output_image, rotate=True):
        image = cv2.imread(path_to_input_image)
        image = self.augment_image(image)
        cv2.imwrite(path_to_output_image, image)

In [3]:
class AugmentationBalancer:
    def __init__(self, random_bound=0.5):
        self.augmentation = Augmentation(random_bound=random_bound)
    
    def fit(self, path_to_folder, delta=5, debug=False):
        self.path_to_folder=path_to_folder        
        directories = os.listdir(path_to_folder)
        real_directories = {}
        self.to_generate = {}
        for directory in directories:
            if os.path.isdir(f"{path_to_folder}/{directory}"):
                count = len(os.listdir(f"{path_to_folder}/{directory}"))
                real_directories[directory]=count
        max_id = max(real_directories, key=real_directories.get)
        print(real_directories)
        self.total_classes = [key for key in real_directories]
        for directory in real_directories:
            diff = real_directories[max_id] - real_directories[directory]
            if(diff>delta):
                self.to_generate[directory]=diff
        if debug:
            print(self.to_generate)
        
    def balance(self, path_to_folder, debug=False):
        for category in self.total_classes:
            source_folder = f"{self.path_to_folder}/{category}"
            destination_folder =f"{path_to_folder}/{category}"
            if os.path.exists(destination_folder) and os.path.isdir(destination_folder):
                shutil.rmtree(destination_folder)
            shutil.copytree(source_folder, destination_folder)
        if debug:
            print(self.to_generate)
        for category in self.to_generate:
            last_id = -1
            for file in os.listdir(f"{path_to_folder}/{category}"):
                id = int(file.split("_")[1].split(".")[0])
                if id > last_id:
                    last_id = id
            for i in range(self.to_generate[category]):
                selected_id = np.random.randint(0, last_id-1) + 1
                if selected_id < 10:
                    selected_id = f"0{selected_id}"
                last_id = last_id + 1
                path_to_selected_image = f"{path_to_folder}/{category}/{category}_{selected_id}.jpg"
                path_to_generate_image = f"{path_to_folder}/{category}/{category}_{last_id}.jpg"
                if debug:
                    print(f"Path to selected image: {path_to_selected_image}")
                    print(f"Path to generate image: {path_to_generate_image}\n")
                self.augmentation.augment_image_file(
                    path_to_selected_image,
                    path_to_generate_image
                )
    
aug_balancer = AugmentationBalancer()
aug_balancer.fit("./final-dataset/imbalanced_flags", debug=True)
aug_balancer.balance("./final-dataset/balanced_flags/", debug=True)

{'A': 400, 'B': 200, 'C': 200, 'D': 200, 'E': 200, 'F': 200, 'G': 200, 'H': 200, 'I': 200, 'J': 200, 'K': 200, 'L': 200, 'M': 200, 'N': 200, 'O': 200, 'P': 200, 'Q': 200, 'R': 200, 'S': 200, 'T': 200, 'U': 200, 'V': 200, 'W': 200, 'X': 200, 'Y': 200, 'Z': 200}
{'B': 200, 'C': 200, 'D': 200, 'E': 200, 'F': 200, 'G': 200, 'H': 200, 'I': 200, 'J': 200, 'K': 200, 'L': 200, 'M': 200, 'N': 200, 'O': 200, 'P': 200, 'Q': 200, 'R': 200, 'S': 200, 'T': 200, 'U': 200, 'V': 200, 'W': 200, 'X': 200, 'Y': 200, 'Z': 200}
{'B': 200, 'C': 200, 'D': 200, 'E': 200, 'F': 200, 'G': 200, 'H': 200, 'I': 200, 'J': 200, 'K': 200, 'L': 200, 'M': 200, 'N': 200, 'O': 200, 'P': 200, 'Q': 200, 'R': 200, 'S': 200, 'T': 200, 'U': 200, 'V': 200, 'W': 200, 'X': 200, 'Y': 200, 'Z': 200}
Path to selected image: ./final-dataset/balanced_flags//B/B_166.jpg
Path to generate image: ./final-dataset/balanced_flags//B/B_201.jpg

Path to selected image: ./final-dataset/balanced_flags//B/B_66.jpg
Path to generate image: ./final-d